# PCA特徴量を追加
- 特徴量グループごとに PCA かける
- それらを特徴量として保存
- (列はいくつ読み込めるのだろう）

In [12]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
%autoreload 2

import os
import sys
import json
import pickle

from numerapi import NumerAPI
from numerai_tools.scoring import numerai_corr, correlation_contribution

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm.auto import tqdm

from scipy.stats import kurtosis, skew

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import IncrementalPCA

sys.path.append("../")

from src.eda import plot_feature_dist
from src.model import MLModel, LightGBMModel
from src.validation import caluculate_metrics
from src.featuring import feature_pca

In [14]:
class cfg:
    dataset_path = "../dataset/v5.0"
    feature_set = "medium"
    threshold_corr = 0.8
    exp = "eda_20250211"

os.makedirs(cfg.exp, exist_ok=True)


In [15]:
with open(f"{cfg.dataset_path}/features.json", "r") as f:
    feature_metadata = json.load(f)

# print(feature_metadata.keys())
feature_sets = feature_metadata["feature_sets"]
# print(feature_sets.keys())
feature_set = feature_sets[cfg.feature_set]
print(f"feature col length: {len(feature_set)}") 
targets = ["target"]

feature col length: 705


In [16]:
feature_groups = [
    "intelligence",
    "charisma",
    "strength",
    "dexterity",
    "constitution",
    "wisdom",
    "agility",
    "serenity",
    "sunshine",
    "rain",
    "midnight",
]
cnt_col = 0
col_set = set([])
for group in feature_groups:
    cnt_col += len(feature_sets[group])
    col_set |= set(feature_sets[group])
print(cnt_col)
print(len(list(col_set)))

2461
2371


In [17]:
drop_cols_d = {}

for feature_group_name in feature_groups:
    print("=" * 64)
    print(feature_group_name)
    feature_set = feature_sets[feature_group_name]
    feature_set = feature_set[:300]
    
    train = pd.read_parquet(
        f"{cfg.dataset_path}/train.parquet",
        columns=["era"] + feature_set + targets
    )
    
    valid = pd.read_parquet(
        f"{cfg.dataset_path}/validation.parquet",
        columns=["era"] + feature_set + targets
        
    )
    
    data = pd.concat([train, valid])
    
    
    correlation_matrix = data[feature_set].corr()
    cols = sorted(correlation_matrix.columns)
    
    high_corr_pairs = [
        (cols[i], cols[j]) 
        for i in range(len(cols))
        for j in range(i + 1, len(cols))
        if abs(correlation_matrix.loc[cols[i], cols[j]]) > cfg.threshold_corr
    ]
    drop_cols = list(set([cols[0] for cols in high_corr_pairs]))
    
    # print(high_corr_pairs)
    # print(drop_cols)
    print(len(high_corr_pairs))
    print(len(drop_cols))
    
    drop_cols_d[feature_group_name] = drop_cols


intelligence
68
28
charisma
600
233
strength
302
109
dexterity
120
42
constitution
754
243
wisdom
367
115
agility
480
121
serenity
137
64
sunshine
542
232
rain
21
19
midnight
332
142


In [18]:
filepath = os.path.join(cfg.exp, "feature_groups_drop_cols.json")
with open(filepath, "w") as f:
    json.dump(drop_cols_d, f, indent=4)

In [19]:
with open(filepath, "r") as f:
    drop_cols_d = json.load(f)

print(len(drop_cols_d.keys()))


cnt = 0


for k, v in drop_cols_d.items():
    print(k)
    print(len(v))
    cnt += len(v)
print("total: ", cnt)

11
intelligence
28
charisma
233
strength
109
dexterity
42
constitution
243
wisdom
115
agility
121
serenity
64
sunshine
232
rain
19
midnight
142
total:  1348


In [12]:
emb_df = feature_pca(data[feature_set], n_components=len(feature_set) // 3)
print(emb_df.shape)

(6324097, 11)
